Notebook esame: 
In questo notebook mi occuperò dell' analisi degli spettri di assorbimento di due specie di molecole (CO2 e C2H2), ottenuti attraverso il metodo 'comb-locked frequency-swept synthesis' (CLFSS), il quale permette di avere simultanaemante un grande livello di precisione e di range di misura in tempi brevi.
Come primo passo importo le librerie necessarie e i dati per un plot preliminare.

In [2]:
# Import libraries
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
#import ipywidgets as widgets
#from ipywidgets import interact_manual
#from ipywidgets import interact
from scipy.signal import find_peaks
import spettro_module as sm

In [3]:
# Dizionario dei dati
dati = {}
bline=[.005,2.5e-9]
# Costruisco i vettori
myList=['C2H2', 'CO2'] #modifica in percorsi relativi
dati['C2H2']=np.loadtxt("ATLAS_2018/C2H2_Pband_10Torr.dat",delimiter="\t", skiprows=2, usecols=range(6))
dati['CO2']=np.loadtxt("ATLAS_2018/CO2_3THz_scan_CRDS__data_2e-2Torr.dat",delimiter="\t", skiprows=2,usecols=range(6))
for i,name in enumerate(myList):
    dati[name][:,-1]=dati[name][:,-1]-bline[i]


In [4]:
#prova plot dati
plt.close('all')

for name in myList:
    plt.figure()
    plt.plot(dati[name][:,0],dati[name][:,-1],label=name)
    plt.legend()
    plt.grid()


Ora come prima analisi è utile trovare la posizione e l'altezza dei picchi, informazioni utili da utilizzare come starting point per il fit dei modelli.

In [5]:
plt.close('all')
#algoritmo di peaks
#per ora lasciamo da parte il calcolo della baseline
peaks={}
hmin=[.02, 6e-9]
width=10
for i,name in enumerate(myList):
    peaks[name]=[dati[name][find_peaks(dati[name][:,-1],height=hmin[i], width=width)[0],0],dati[name][find_peaks(dati[name][:,-1],height=hmin[i], width=width)[0],-1]]
    plt.figure()
    plt.plot(dati[name][:,0],dati[name][:,-1])
    plt.plot(peaks[name][0],peaks[name][1], '.')
    plt.grid()


Ora si può procedere all' analisi in due metodi, che chiameremo 'fit dei picchi' e 'fit delle bande'. Il primo consiste nell'analizzare con un fit i singoli picchi di assorbimento, mentre il secondo si procede con l'analisi dei massimi appartenenti alla stessa banda. Iniziamo col primo metodo.

In [6]:
#fit dei picchi
plt.close('all')


ctrlv=[[],[]]
for i,name in enumerate(myList):
    #i=1
    #name='CO2'
    plt.figure()
    plt.grid()
    cond=(dati[name][:,-1]<=0)
    dati2=dati[name][:,0]*cond
    plt.plot(dati[name][:,0],dati[name][:,-1],'k',linewidth=.1)
    for c,j in enumerate(peaks[name][0]):
        #perche` questa formula? voglio trovare il pt piu` vicino che sia sotto la bline (altrimenti arrivo al picco)`
        xmin=np.argmin(1/(dati2-j))
        xmax=np.argmax(1/(dati2-j)) 
        xfit=dati[name][xmin:xmax,0]
        yfit=dati[name][xmin:xmax,-1]
        #grazie all'if elimino i picchi sovrapposti
        if c!=len(peaks[name][0])-1:
            if np.logical_and(peaks[name][0][c+1]>xfit[-1],peaks[name][0][c-1]<xfit[0]):
                
                FWHM=xfit[yfit>=max(yfit)/2][-1]-xfit[yfit>=max(yfit)/2][0]

                #per il fit utilizzo la funzione di voigt
                p0=[np.dot(np.diff(xfit),yfit[:-1]),FWHM/(2*np.sqrt(2*np.log(2))),j,FWHM/2]
                fit,cov=curve_fit(sm.voigt,xfit,yfit,p0=p0,
                                  bounds=((p0[0],p0[1]/2,p0[2]-.1,p0[3]*.09),(p0[0]*2,p0[1],p0[2]+.1,p0[3]*2)))
                #grazie a questo vettore controllo la bonta` del guess
                ctrlv[i].append((p0-fit)/p0)
                #print((p0-fit)/p0)
                #controllo che funzioni il metodo scelto
                #plt.plot(dati[name][:,0],1/(dati2-j))
                plt.plot(xfit,yfit,'r',linewidth=1)
                plt.plot(xfit,sm.voigt(xfit,*fit),'--b',linewidth=.5)
    
    #nu Rolex or e diamant
    plt.xlabel('Frequenza [THz]')
    plt.ylabel('Assorbimento')
    plt.title(name)
    plt.legend(['whole profile','selected data','fit curve'])


In [7]:
#provo a plottare i massimi dei massimi e vedere se riconosce le bande
plt.close('all')
peaks2={}
hmin=[.15, 2e-8]
width=0
gmu=[0,0]
dist=[7,2] #,distance=dist
for i,name in enumerate(myList):
    peaks2[name]=[peaks[name][0][find_peaks(peaks[name][1],height=hmin[i],distance=dist[i])[0]],peaks[name][1][find_peaks(peaks[name][1],height=hmin[i],distance=dist[i])[0]]]
    #e` anche possibile dare cosi` una stima del centro delle bande
    gmu[i]=peaks2[name][0][find_peaks(-peaks2[name][1])[0]]
    plt.figure()
    plt.plot(peaks[name][0],peaks[name][1], 'r-*')
    plt.plot(peaks2[name][0],peaks2[name][1], 'b.')
    plt.plot(gmu[i],0,'g.')
    plt.grid()


Illustriamo ora il secondo metodo, ovvero il fit delle bande attraverso la funzione di maxwell-boltzmann. La difficolta` qui sta nel distinguere le varie bande, trovando un metodo che possa farlo in autonomia

In [8]:
#fit delle bande 
plt.close('all')
kb=1.380649 *10**-23
#gmu=[196.8,190.29]
eps=[2e-1,1e-8]

vmax=[0,0]
norm=[0,0]

for i,name in enumerate(myList):
    xb=np.copy(peaks[name][0])
    yb=np.copy(peaks[name][1])
    #qui potrei inserire un ciclo for per fittare ogni banda
    b_max=np.argmax(yb)
    vmax[i]=xb[b_max]-gmu[i]
    #vmax e` possibile trovarlo anche conoscendo T e m
    #vmax[1]=np.sqrt(kb*(8.21917808*10**21)) 
    #f(vmax)=C*... inverto la relazione
    norm[i]=np.exp(1)*yb[b_max]*abs(vmax[i])*(np.pi**(1/2))/4
    #si puo` provare anche approssimando ad un triangolo
    
    #guess plot
    plt.figure()
    plt.grid()
    plt.plot(xb,yb, '.')
    plt.plot(xb,sm.max_boltz(xb,vmax[i],norm[i],gmu[i])) #(xb[b_max]-gmu[i]),norm
    bcon=(abs(yb-sm.max_boltz(xb,vmax[i],norm[i],gmu[i])))*abs(xb-xb[b_max])<eps[i] #*abs(xb-xb[b_max])
    #bcon=[for j in len(xb[bcon]):  ]
    plt.plot(xb[bcon],yb[bcon],'.r')
    


In [9]:
#ora posso provare ad unire i metodi delle 2 celle precedenti
plt.close('all')
#idea: ciclo while che diminuisce eps ma lo tiene tale che tutti i selected point siano nella ricostruzione
#oppure: soglia in altezza
eps=[2e-1,3e-8]

for i,name in enumerate(myList):
    #qui potrei inserire un ciclo per fittare ogni banda
    b_max=np.argmax(peaks2[name][1])
    if peaks2[name][0][b_max]>gmu[i]:
        cond2=peaks2[name][0]>gmu[i]
        cond3=peaks[name][0]>gmu[i]
    else:
        cond2=peaks2[name][0]<gmu[i]
        cond3=peaks[name][0]<gmu[i]
    xb=np.copy(peaks2[name][0][cond2])
    yb=np.copy(peaks2[name][1][cond2])
    vmax[i]=xb[np.argmax(yb)]-gmu[i]
    #vmax e` possibile trovarlo anche conoscendo T e m
    #vmax[1]=np.sqrt(kb*(8.21917808*10**21)) 
    #f(vmax)=C*... inverto la relazione
    norm[i]=np.exp(1)*yb[np.argmax(yb)]*abs(vmax[i])*(np.pi**(1/2))/4
    #si puo` provare anche approssimando ad un triangolo
    p0=[vmax[i],norm[i],gmu[i]]
    fit,cov=curve_fit(sm.max_boltz,xb,yb,p0=p0)
    #ricostruzione banda #*abs(xb-xb[np.argmax(yb)]) #*abs(xb-xb[b_max])  USA hmin[i]
    #bcon=[for j in len(xb[bcon]):  ]
    bcon=((abs(peaks[name][1]-sm.max_boltz(peaks[name][0],*fit)))<eps[i])*cond3*(peaks[name][1]>=hmin[i])
    fit2,cov2=curve_fit(sm.max_boltz,peaks[name][0][bcon],peaks[name][1][bcon],p0=p0)
    #plot
    plt.figure()
    plt.grid()
    plt.plot(xb,yb, '*',markersize=7)
    plt.plot(peaks[name][0],peaks[name][1],'.',markersize=2)
    plt.plot(xb,sm.max_boltz(xb,vmax[i],norm[i],gmu[i]),linewidth=.5)
    plt.plot(xb,sm.max_boltz(xb,*fit),linewidth=.5)
    plt.plot(peaks[name][0][bcon],peaks[name][1][bcon],'.r')
    plt.plot(peaks[name][0][bcon],sm.max_boltz(peaks[name][0][bcon],*fit2),linewidth=1)
    plt.legend(['selected data','whole data','guess','1st fit','reconstructed band','2nd fit'])
    #nu Rolex or e diamant
    plt.xlabel('Frequenza [THz]')
    plt.ylabel('Assorbimento')
    plt.title(name)

In [23]:
#all in one
plt.close('all')

eps=[2e-1,3e-8]
vmax=[0,0]
norm=[0,0]
peaks2={}
hmin=[.15, 2e-8]
width=0
gmu=[0,0]
dist=[7,2]

for i,name in enumerate(myList):
    #qui potrei inserire un ciclo per fittare ogni banda
    #for j in range(#bande):
    peaks2[name]=[peaks[name][0][find_peaks(peaks[name][1],height=hmin[i],distance=dist[i])[0]],peaks[name][1][find_peaks(peaks[name][1],height=hmin[i],distance=dist[i])[0]]]
    #e` anche possibile dare cosi` una stima del centro delle bande
    gmu[i]=peaks2[name][0][find_peaks(-peaks2[name][1])[0]]
    b_max=np.argmax(peaks2[name][1])
    plt.figure()
    plt.plot(peaks[name][0],peaks[name][1],'k.',markersize=2)
    for k in range(2):
        if peaks2[name][0][b_max]>gmu[i]:
            cond2=peaks2[name][0]>=gmu[i]
            cond3=peaks[name][0]>=gmu[i]
        else:
            cond2=peaks2[name][0]<=gmu[i]
            cond3=peaks[name][0]<=gmu[i]
        #lasciamo perdere l'approccio di matematica booleana
        if np.logical_and(i!=0,k==1):
            cond2=np.logical_not(cond2)
            cond3=np.logical_not(cond3)
        xb=np.copy(peaks2[name][0][cond2])
        yb=np.copy(peaks2[name][1][cond2])
        vmax[i]=abs(xb[np.argmax(yb)]-gmu[i])
        #vmax e` possibile trovarlo anche conoscendo T e m
        #vmax[1]=np.sqrt(kb*(8.21917808*10**21)) 
        #f(vmax)=C*... inverto la relazione
        norm[i]=np.exp(1)*yb[np.argmax(yb)]*abs(vmax[i])*(np.pi**(1/2))/4
        #si puo` provare anche approssimando ad un triangolo
        p0=[vmax[i],norm[i],gmu[i]]
        fit,cov=curve_fit(sm.max_boltz,xb,yb,p0=p0)
        #si potrebbe computare eps partendo ad esempio dai residui o dalla std, ho scelto 2.5 volte la std
        eps[i]=2.5*np.sqrt(np.mean((yb-sm.max_boltz(xb,*fit))**2))
        #ricostruzione banda #*abs(xb-xb[np.argmax(yb)]) #*abs(xb-xb[b_max])  
        #bcon=[for j in len(xb[bcon]):  ]
        bcon=((abs(peaks[name][1]-sm.max_boltz(peaks[name][0],*fit)))<=eps[i])*cond3*(peaks[name][1]>=hmin[i])
        fit2,cov2=curve_fit(sm.max_boltz,peaks[name][0][bcon],peaks[name][1][bcon],p0=p0)
        #plot
        plt.plot(xb,yb, '*b',markersize=7)
        plt.plot(peaks[name][0][cond3],sm.max_boltz(peaks[name][0][cond3],vmax[i],norm[i],gmu[i]),'g',linewidth=.5)
        plt.plot(peaks[name][0][cond3],sm.max_boltz(peaks[name][0][cond3],*fit),'grey',linewidth=.5)
        plt.plot(peaks[name][0][bcon],peaks[name][1][bcon],'.r')
        plt.plot(peaks[name][0][cond3],sm.max_boltz(peaks[name][0][cond3],*fit2),'k--',linewidth=1)

    #nu Rolex or e diamant
    plt.grid()
    plt.legend(['whole data','selected data','guess','1st fit','reconstructed band','2nd fit'])
    plt.xlabel('Frequenza [THz]')
    plt.ylabel('Assorbimento')
    plt.title(name)